In [ ]:
import pandas as pd
import numpy as np

pr_gen_fuel = pd.read_parquet("../data/raw_eia923__puerto_rico_generation_fuel.parquet")
pr_plant_frame = pd.read_parquet("../data/raw_eia923__puerto_rico_plant_frame.parquet")

In [ ]:
# Handle EIA null values
pr_gen_fuel = pr_gen_fuel.replace(to_replace = ".", value = pd.NA)

# Silence some warnings about deprecated Pandas behavior
pd.set_option("future.no_silent_downcasting", True)

# Convert data types (mmbtu/units to numeric, booleans, categories)
pr_gen_fuel = pr_gen_fuel.convert_dtypes()
for colname in pr_gen_fuel.columns:
    if (
        "fuel_consumption" in colname
        or "fuel_consumed" in colname
        or "net_generation" in colname
        or "fuel_mmbtu_per_unit" in colname
    ):
        pr_gen_fuel[colname] = pr_gen_fuel[colname].astype("float64")


pr_gen_fuel["associated_combined_heat_power"] = (
    pr_gen_fuel["associated_combined_heat_power"]
    .astype("object") # necessary for the types to work for the .replace() call
    .replace({"Y": True, "N": False})
    .astype("boolean")
)
pr_gen_fuel = pr_gen_fuel.astype({
    "energy_source_code": "category",
    "fuel_type_code_agg": "category",
    "prime_mover_code": "category",
    "reporting_frequency_code": "category",
    "data_maturity": "category",
    "plant_state": "category"
})

In [ ]:
# Handle EIA null values
pr_plant_frame = pr_plant_frame.replace(to_replace = ".", value = pd.NA)

# Convert data types (mmbtu/units to numeric, categories, booleans)
pr_plant_frame = pr_plant_frame.convert_dtypes()
pr_plant_frame["associated_combined_heat_power"] = (
    pr_plant_frame["associated_combined_heat_power"]
    .astype("object") # necessary for the types to work for the .replace() call
    .replace({"Y": True, "N": False})
    .astype("boolean")
)
pr_plant_frame = pr_plant_frame.astype({
    "reporting_frequency_code": "category",
    "data_maturity": "category",
    "plant_state": "category"
})

pr_plant_frame.info()

In [ ]:
#### monthly pivoting

# set up shared index
index_cols = ["plant_id_eia", "plant_name_eia", "report_year", "prime_mover_code", "energy_source_code", "fuel_unit"]


In [ ]:
# Pivot fuel_consumed_for_electricity MMBTU columns

fuel_elec_mmbtu_cols = index_cols + [col for col in pr_gen_fuel.columns if "fuel_consumed_for_electricity_mmbtu" in col]
fuel_elec_mmbtu = pr_gen_fuel.loc[:, fuel_elec_mmbtu_cols]

## Melt the fuel_consumed columns
fuel_elec_mmbtu_melt = fuel_elec_mmbtu.melt(
    id_vars=index_cols,
    var_name="month",
    value_name="fuel_consumed_for_electricity_mmbtu"
)
fuel_elec_mmbtu_melt["month"] = fuel_elec_mmbtu_melt["month"].str.replace("fuel_consumed_for_electricity_mmbtu_", "")
fuel_elec_mmbtu_melt = fuel_elec_mmbtu_melt.set_index(index_cols + ["month"])
fuel_elec_mmbtu_melt

In [ ]:
# Pivot fuel_consumed_for_electricity UNITS columns
fuel_elec_units_cols = index_cols + [col for col in pr_gen_fuel.columns if "fuel_consumed_for_electricity_units" in col]
fuel_elec_units = pr_gen_fuel.loc[:, fuel_elec_units_cols]

## Melt the fuel_consumed columns
fuel_elec_units_melt = fuel_elec_units.melt(
    id_vars=index_cols,
    var_name="month",
    value_name="fuel_consumed_for_electricity_units"
)
fuel_elec_units_melt["month"] = fuel_elec_units_melt["month"].str.replace("fuel_consumed_for_electricity_units_", "")
fuel_elec_units_melt = fuel_elec_units_melt.set_index(index_cols + ["month"])
fuel_elec_units_melt

In [ ]:
# Pivot fuel_consumed MMBTU columns

fuel_mmbtu_cols = index_cols + [col for col in pr_gen_fuel.columns if "fuel_consumed_mmbtu" in col]
fuel_mmbtu = pr_gen_fuel.loc[:, fuel_mmbtu_cols]

## Melt the fuel_consumed columns
fuel_mmbtu_melt = fuel_mmbtu.melt(
    id_vars=index_cols,
    var_name="month",
    value_name="fuel_consumed_mmbtu"
)
fuel_mmbtu_melt["month"] = fuel_mmbtu_melt["month"].str.replace("fuel_consumed_mmbtu_", "")
fuel_mmbtu_melt = fuel_mmbtu_melt.set_index(index_cols + ["month"])
fuel_mmbtu_melt

In [ ]:
# Pivot fuel_consumed UNITS columns

fuel_units_cols = index_cols + [col for col in pr_gen_fuel.columns if "fuel_consumed_units" in col]
fuel_units = pr_gen_fuel.loc[:, fuel_units_cols]

## Melt the fuel_consumed columns
fuel_units_melt = fuel_units.melt(
    id_vars=index_cols,
    var_name="month",
    value_name="fuel_consumed_units"
)
fuel_units_melt["month"] = fuel_units_melt["month"].str.replace("fuel_consumed_units_", "")
fuel_units_melt = fuel_units_melt.set_index(index_cols + ["month"])
fuel_units_melt

In [ ]:
# Pivot net_generation columns

net_gen_cols = index_cols + [col for col in pr_gen_fuel.columns if col.startswith("net_generation_mwh")]
net_gen = pr_gen_fuel.loc[:, net_gen_cols]

## Melt the fuel_consumed columns
net_gen_melt = net_gen.melt(
    id_vars=index_cols,
    var_name="month",
    value_name="net_generation_mwh"
)
net_gen_melt["month"] = net_gen_melt["month"].str.replace("net_generation_mwh_", "")
net_gen_melt = net_gen_melt.set_index(index_cols + ["month"])
net_gen_melt

In [ ]:
pr_gen_fuel_melt = pd.concat(
    [fuel_elec_mmbtu_melt, fuel_elec_units_melt, fuel_mmbtu_melt, fuel_units_melt, net_gen_melt],
    axis="columns",
).reset_index()
pr_gen_fuel_melt

In [ ]:
## Create date from month and year
pr_gen_fuel_melt["date"] = pd.to_datetime(
    pr_gen_fuel_melt["month"] + pr_gen_fuel_melt["report_year"].astype(str),
    format="%B%Y",
)
## Drop old date columns
pr_gen_fuel_clean = pr_gen_fuel_melt.drop(columns = ["report_year", "month"])

In [ ]:
## Drop a bad plant
pr_gen_fuel_final = pr_gen_fuel_clean.loc[
    ~((pr_gen_fuel_clean.plant_id_eia == 62410) 
    & (pr_gen_fuel_clean.date.dt.year == 2020)
    & (pr_gen_fuel_clean.fuel_consumed_for_electricity_mmbtu.isnull()))
]

# drop after 2025-03-01 (for now)
pr_gen_fuel_final = pr_gen_fuel_final.loc[pr_gen_fuel_clean.date < pd.Timestamp("2025-03-01")]


In [ ]:
## some investigation into NA values
clean_index_cols = ["date", "energy_source_code", "prime_mover_code", "plant_id_eia", "plant_name_eia", "fuel_unit"]
pp = pr_gen_fuel_clean.set_index(clean_index_cols).isna().all(axis=1).reset_index().rename(columns={0: "isna"}).loc[:, ["date", "plant_name_eia", "isna"]]

pp.plot.scatter(x="date", y="plant_name_eia", c=pp["isna"].astype(int), colormap="viridis", s=10, alpha=0.2, figsize=(10, 20))

# it turns out that after 2025-03-01 there's a pile of NAs, and many plants have multiple generators, some of which report as all-NA while their sibling generators are reporting non-NA values
# though Aguirre Plant and Hewlett Packard Puerto Rico seem to have actual stretches of all-NA time that might be wroth dropping.

In [ ]:
### make a list of all the monthly columns
import calendar
monthly_columns = []
for col in pr_gen_fuel.columns:
    for month in calendar.Month:
        if col.endswith(month.name.lower()):
            monthly_columns.append(col)

In [ ]:
### annual table
pr_gen_fuel_annual = pr_gen_fuel.drop(columns=monthly_columns)

In [ ]:
### output
pr_gen_fuel_final.to_parquet("../data/pr_gen_fuel_monthly.parquet")
pr_gen_fuel_annual.to_parquet("../data/pr_gen_fuel_annual.parquet")
pr_plant_frame.to_parquet("../data/pr_plant_frame.parquet")